In [107]:
import paddlehub as hub
import paddle
import os,io,csv
from paddlehub.datasets.base_nlp_dataset import InputExample,TextClassificationDataset
#查看paddle版本
print("paddle版本:"+paddle.__version__)
#查看paddlehub版本
print("paddlehub版本:"+hub.__version__)
!hub show ernie




paddle版本:2.0.1
paddlehub版本:2.0.4
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead o

In [108]:
#解压数据集输出，输出验证集前三条、后三条数据（方法1）
%cd /home/aistudio/data/data80226/

print("---------验证集前三条数据----------")
!head -n 4 thu_news/valid.txt
print("---------验证集后三条数据----------")
!tail -n 3 thu_news/valid.txt

/home/aistudio/data/data80226
---------验证集前三条数据----------
text_a	label
直击特里勾手助小牛反超神鸟发威火箭仍处劣势　　新浪体育讯　北京时间4月16日消息，火箭今天迎来常规赛的收官战。客场挑战达拉斯小牛的比赛将关系到火箭队最终的排名，目前西部的竞争仍然非常激励。尤其是西部第二到第七的六支球队将在今天展开捉对厮杀的情况下，黄蜂、小牛、开拓者以及马刺都有可能是火箭队的下一个对手。以下为本场比赛的精彩瞬间——　　第四节比赛还剩下8分多钟，姚明重新回到场上，但是小牛队的霍里随后在右翼接队友传球后上演一记三分跳投，虽然身体动作已经变形，但是他仍然将球命中，随后基德在弧顶处的一记三分跳投帮助小牛终于将比分反超，火箭在进攻中直接打不开局面，而小牛更是利用特里空切后的一记勾手将比分反超至4分，包括库班在内的所有小牛现场球迷都站起来振臂高呼，而此时阿德尔曼还在场边低头深沉的思索中。　　比赛还剩下最后5分多钟，洛瑞带球突破中被小牛队员犯规，洛瑞来不及刹车让自己直接撞到了观众席上，右腿疼痛难忍的他脸都几乎变形，结果他两罚两中，火箭队还落后两分。随后，洛瑞将球直接传给此时已经回到场上的阿泰，野兽此时将球稳下，并没有急于进攻，随后他顺势将球塞给兰德里，神鸟利用对方站位空隙直接杀到篮下，上篮成功，火箭将比分终于追至80平。场上局面对于火箭来说绝对是不容有失。　　(sabrina)	体育
北京网购年消费额112亿元　　商报讯(记者吴文治)昨日，淘宝网发布的《2009-2010年度中国网购热门城市报告》显示，北京年度网购消费力达112.5亿元，与上海相差近62亿元，位列十大热门消费力城市第二位。此外，男性网购的消费金额高出女性，与“女性是网购主力军”的传统观念不符。　　淘宝公布的报告显示，中国网购消费力十大城市分别是上海、北京、深圳、杭州、广州、南京、苏州、天津、温州和宁波。主要集中在以江浙沪为主的长三角地区、以广深为主的珠三角地区和以北京为主的京津地区。北京年度网购112.5亿元的消费额，占国内城市网购消费额的5.6%。　　中国网购消费力十大城市的消费金额性别来源比例中，男性占比超过了女性。前者占比达到53.5%，后者则为46.5%。不过，在成交人数、成交笔数等关键数据上显示，女性消费者均高于男性。此外，在十大网购热门城市中，2

In [109]:

##加载数据集
class ThuNews(TextClassificationDataset):
    base_path="/home/aistudio/data/data80226/thu_news"
    
    def __init__(self, tokenizer, mode='train', max_seq_len=128):
        if mode == 'train':
            data_file = 'train.txt'
        elif mode == 'test':
            data_file = 'test.txt'
        else:
            data_file = 'valid.txt'
        super(ThuNews, self).__init__(
            base_path=self.base_path,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            data_file=data_file,
            is_file_with_header=True,
            label_list=['体育', '科技', '社会', '娱乐', '股票', '房产', '教育', '时政', '财经', '星座', '游戏', '家居', '彩票', '时尚'])
            
            
    # 解析文本文件里的样本
    def _read_file(self, input_file, is_file_with_header: bool = False):
        if not os.path.exists(input_file):
            raise RuntimeError("The file {} is not found.".format(input_file))
        else:
            with io.open(input_file, "r", encoding="UTF-8") as f:
                reader = csv.reader(f, delimiter="\t", quotechar=None)
                examples = []
                seq_id = 0
                header = next(reader) if is_file_with_header else None
                for line in reader:
                    example = InputExample(guid=seq_id, text_a=line[0], label=line[1])
                    seq_id += 1
                    examples.append(example)
                return examples

model = hub.Module(name="ernie", task='seq-cls', num_classes=14)
#开始训练模型
train_dataset = ThuNews(model.get_tokenizer(), mode='train', max_seq_len=128)
valid_dataset = ThuNews(model.get_tokenizer(), mode='valid', max_seq_len=128)
test_dataset = ThuNews(model.get_tokenizer(), mode='test', max_seq_len=128)


#输出验证集前三条、后三条数据（方法2）
#前三条
for e in valid_dataset.examples[:3]:
    print("---------验证集前三条数据----------")
    print(e)            
#后三条
n=len(valid_dataset)
for e in valid_dataset.examples[n-3:]:
    print("---------验证集后三条数据----------")
    print(e)



[2021-04-13 17:36:19,255] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-04-13 17:36:21,081] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2021-04-13 17:37:43,072] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2021-04-13 17:37:56,358] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


---------验证集前三条数据----------
text=直击特里勾手助小牛反超神鸟发威火箭仍处劣势　　新浪体育讯　北京时间4月16日消息，火箭今天迎来常规赛的收官战。客场挑战达拉斯小牛的比赛将关系到火箭队最终的排名，目前西部的竞争仍然非常激励。尤其是西部第二到第七的六支球队将在今天展开捉对厮杀的情况下，黄蜂、小牛、开拓者以及马刺都有可能是火箭队的下一个对手。以下为本场比赛的精彩瞬间——　　第四节比赛还剩下8分多钟，姚明重新回到场上，但是小牛队的霍里随后在右翼接队友传球后上演一记三分跳投，虽然身体动作已经变形，但是他仍然将球命中，随后基德在弧顶处的一记三分跳投帮助小牛终于将比分反超，火箭在进攻中直接打不开局面，而小牛更是利用特里空切后的一记勾手将比分反超至4分，包括库班在内的所有小牛现场球迷都站起来振臂高呼，而此时阿德尔曼还在场边低头深沉的思索中。　　比赛还剩下最后5分多钟，洛瑞带球突破中被小牛队员犯规，洛瑞来不及刹车让自己直接撞到了观众席上，右腿疼痛难忍的他脸都几乎变形，结果他两罚两中，火箭队还落后两分。随后，洛瑞将球直接传给此时已经回到场上的阿泰，野兽此时将球稳下，并没有急于进攻，随后他顺势将球塞给兰德里，神鸟利用对方站位空隙直接杀到篮下，上篮成功，火箭将比分终于追至80平。场上局面对于火箭来说绝对是不容有失。　　(sabrina)	label=体育
---------验证集前三条数据----------
text=北京网购年消费额112亿元　　商报讯(记者吴文治)昨日，淘宝网发布的《2009-2010年度中国网购热门城市报告》显示，北京年度网购消费力达112.5亿元，与上海相差近62亿元，位列十大热门消费力城市第二位。此外，男性网购的消费金额高出女性，与“女性是网购主力军”的传统观念不符。　　淘宝公布的报告显示，中国网购消费力十大城市分别是上海、北京、深圳、杭州、广州、南京、苏州、天津、温州和宁波。主要集中在以江浙沪为主的长三角地区、以广深为主的珠三角地区和以北京为主的京津地区。北京年度网购112.5亿元的消费额，占国内城市网购消费额的5.6%。　　中国网购消费力十大城市的消费金额性别来源比例中，男性占比超过了女性。前者占比达到53.5%，后者则为46.5%。不过，在成交人数、成交笔数等关键数据上显示，女性消费者均高于男性。此外，在十大网购热门城市中，

In [110]:

optimizer = paddle.optimizer.Adam(learning_rate=5e-5, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='./ckpt', use_gpu=True)
trainer.train(train_dataset, epochs=3, batch_size=32, eval_dataset=valid_dataset, save_interval=1)   # 配置训练参数，启动训练，并指定验证集



[2021-04-13 17:38:10,816] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...
[2021-04-13 17:38:13,042] [   TRAIN] - Epoch=1/3, Step=10/282 loss=2.2089 acc=0.3219 lr=0.000050 step/sec=4.50 | ETA 00:03:08
[2021-04-13 17:38:15,194] [   TRAIN] - Epoch=1/3, Step=20/282 loss=1.5204 acc=0.6031 lr=0.000050 step/sec=4.65 | ETA 00:03:05
[2021-04-13 17:38:17,344] [   TRAIN] - Epoch=1/3, Step=30/282 loss=1.1498 acc=0.7000 lr=0.000050 step/sec=4.65 | ETA 00:03:03
[2021-04-13 17:38:19,503] [   TRAIN] - Epoch=1/3, Step=40/282 loss=0.8613 acc=0.7906 lr=0.000050 step/sec=4.63 | ETA 00:03:03
[2021-04-13 17:38:21,655] [   TRAIN] - Epoch=1/3, Step=50/282 loss=0.6544 acc=0.8375 lr=0.000050 step/sec=4.65 | ETA 00:03:03
[2021-04-13 17:38:23,810] [   TRAIN] - Epoch=1/3, Step=60/282 loss=0.6264 acc=0.8438 lr=0.000050 step/sec=4.64 | ETA 00:03:03
[2021-04-13 17:38:25,969] [   TRAIN] - Epoch=1/3, Step=70/282 loss=0.5065 acc=0.8906 lr=0.000050 step/sec=4.63 | ETA 00:03:03
[2021-04-13 17:38:2

In [111]:
result = trainer.evaluate(test_dataset, batch_size=32) 

[2021-04-13 17:42:30,632] [    INFO] - Evaluation on validation dataset: \
[2021-04-13 17:42:34,132] [    EVAL] - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation o

In [112]:
# 开始预测
data = [
    # 房产
    ["昌平京基鹭府10月29日推别墅1200万套起享97折　　新浪房产讯(编辑郭彪)京基鹭府(论坛相册户型样板间点评地图搜索)售楼处位于昌平区京承高速北七家出口向西南公里路南。项目预计10月29日开盘，总价1200万元/套起，2012年年底入住。待售户型为联排户型面积为410-522平方米，独栋户型面积为938平方米，双拼户型面积为522平方米。　　京基鹭府项目位于昌平定泗路与东北路交界处。项目周边配套齐全，幼儿园：伊顿双语幼儿园、温莎双语幼儿园；中学：北师大亚太实验学校、潞河中学(北京市重点)；大学：王府语言学校、北京邮电大学、现代音乐学院；医院：王府中西医结合医院(三级甲等)、潞河医院、解放军263医院、安贞医院昌平分院；购物：龙德广场、中联万家商厦、世纪华联超市、瑰宝购物中心、家乐福超市；酒店：拉斐特城堡、鲍鱼岛；休闲娱乐设施：九华山庄、温都温泉度假村、小汤山疗养院、龙脉温泉度假村、小汤山文化广场、皇港高尔夫、高地高尔夫、北鸿高尔夫球场；银行：工商银行、建设银行、中国银行、北京农村商业银行；邮局：中国邮政储蓄；其它：北七家建材城、百安居建材超市、北七家镇武装部、北京宏翔鸿企业孵化基地等，享受便捷生活。"],
    # 游戏
    ["尽管官方到今天也没有公布《使命召唤：现代战争2》的游戏详情，但《使命召唤：现代战争2》首部包含游戏画面的影片终于现身。虽然影片仅有短短不到20秒，但影片最后承诺大家将于美国时间5月24日NBA职业篮球东区决赛时将会揭露更多的游戏内容。　　这部只有18秒的广告片闪现了9个镜头，能够辨识的场景有直升机飞向海岛军事工事，有飞机场争夺战，有潜艇和水下工兵，有冰上乘具，以及其他的一些镜头。整体来看《现代战争2》很大可能仍旧与俄罗斯有关。　　片尾有一则预告：“May24th，EasternConferenceFinals”，这是什么？这是说当前美国NBA联赛东部总决赛的日期。原来这部视频是NBA季后赛奥兰多魔术对波士顿凯尔特人队时，TNT电视台播放的广告。"],
    # 体育
    ["罗马锋王竟公然挑战两大旗帜拉涅利的球队到底错在哪　　记者张恺报道主场一球小胜副班长巴里无可吹捧，罗马占优也纯属正常，倒是托蒂罚失点球和前两号门将先后受伤(多尼以三号身份出场)更让人揪心。阵容规模扩大，反而表现不如上赛季，缺乏一流强队的色彩，这是所有球迷对罗马的印象。　　拉涅利说：“去年我们带着嫉妒之心看国米，今年我们也有了和国米同等的超级阵容，许多教练都想有罗马的球员。阵容广了，寻找队内平衡就难了，某些时段球员的互相排斥和跟从前相比的落差都正常。有好的一面，也有不好的一面，所幸，我们一直在说一支伟大的罗马，必胜的信念和够级别的阵容，我们有了。”拉涅利的总结由近一阶段困扰罗马的队内摩擦、个别球员闹意见要走人而发，本赛季技术层面强化的罗马一直没有上赛季反扑的面貌，内部变化值得球迷关注。"],
    # 教育
    ["新总督致力提高加拿大公立教育质量　　滑铁卢大学校长约翰斯顿先生于10月1日担任加拿大总督职务。约翰斯顿先生还曾任麦吉尔大学长，并曾在多伦多大学、女王大学和西安大略大学担任教学职位。　　约翰斯顿先生在就职演说中表示，要将加拿大建设成为一个“聪明与关爱的国度”。为实现这一目标，他提出三个支柱：支持并关爱家庭、儿童；鼓励学习与创造；提倡慈善和志愿者精神。他尤其强调要关爱并尊重教师，并通过公立教育使每个人的才智得到充分发展。"]
]

label_list=['体育', '科技', '社会', '娱乐', '股票', '房产', '教育', '时政', '财经', '星座', '游戏', '家居', '彩票', '时尚']
label_map = { 
    idx: label_text for idx, label_text in enumerate(label_list)
}

model = hub.Module(
    name='ernie',
    task='seq-cls',
    load_checkpoint='./ckpt/best_model/model.pdparams',
    label_map=label_map)
results = model.predict(data, max_seq_len=128, batch_size=1, use_gpu=True)
for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text[0], results[idx]))

[2021-04-13 17:42:34,147] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-04-13 17:42:39,562] [    INFO] - Loaded parameters from /home/aistudio/data/data80226/ckpt/best_model/model.pdparams
[2021-04-13 17:42:39,704] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


Data: 昌平京基鹭府10月29日推别墅1200万套起享97折　　新浪房产讯(编辑郭彪)京基鹭府(论坛相册户型样板间点评地图搜索)售楼处位于昌平区京承高速北七家出口向西南公里路南。项目预计10月29日开盘，总价1200万元/套起，2012年年底入住。待售户型为联排户型面积为410-522平方米，独栋户型面积为938平方米，双拼户型面积为522平方米。　　京基鹭府项目位于昌平定泗路与东北路交界处。项目周边配套齐全，幼儿园：伊顿双语幼儿园、温莎双语幼儿园；中学：北师大亚太实验学校、潞河中学(北京市重点)；大学：王府语言学校、北京邮电大学、现代音乐学院；医院：王府中西医结合医院(三级甲等)、潞河医院、解放军263医院、安贞医院昌平分院；购物：龙德广场、中联万家商厦、世纪华联超市、瑰宝购物中心、家乐福超市；酒店：拉斐特城堡、鲍鱼岛；休闲娱乐设施：九华山庄、温都温泉度假村、小汤山疗养院、龙脉温泉度假村、小汤山文化广场、皇港高尔夫、高地高尔夫、北鸿高尔夫球场；银行：工商银行、建设银行、中国银行、北京农村商业银行；邮局：中国邮政储蓄；其它：北七家建材城、百安居建材超市、北七家镇武装部、北京宏翔鸿企业孵化基地等，享受便捷生活。 	 Lable: 房产
Data: 尽管官方到今天也没有公布《使命召唤：现代战争2》的游戏详情，但《使命召唤：现代战争2》首部包含游戏画面的影片终于现身。虽然影片仅有短短不到20秒，但影片最后承诺大家将于美国时间5月24日NBA职业篮球东区决赛时将会揭露更多的游戏内容。　　这部只有18秒的广告片闪现了9个镜头，能够辨识的场景有直升机飞向海岛军事工事，有飞机场争夺战，有潜艇和水下工兵，有冰上乘具，以及其他的一些镜头。整体来看《现代战争2》很大可能仍旧与俄罗斯有关。　　片尾有一则预告：“May24th，EasternConferenceFinals”，这是什么？这是说当前美国NBA联赛东部总决赛的日期。原来这部视频是NBA季后赛奥兰多魔术对波士顿凯尔特人队时，TNT电视台播放的广告。 	 Lable: 游戏
Data: 罗马锋王竟公然挑战两大旗帜拉涅利的球队到底错在哪　　记者张恺报道主场一球小胜副班长巴里无可吹捧，罗马占优也纯属正常，倒是托蒂罚失点球和前两号门将先后受伤(多尼以三号身份出场)更让人揪心。阵容规模扩大，反而表现不如上赛季，缺乏一流强队的色彩，这是